# Calculating DCJ distance between two unsigned, circular and/or linear genomes with no duplicates

Credit:
- https://github.com/mlliou112/py-dcj (python version)
- ChatGPT for converting to Julia 

In [1]:
using Base.Iterators
using Parameters

abstract type AbstractGene end

In [36]:
"""
Representation for a telomere in DCJ model
"""

@with_kw struct Telomere <: AbstractGene
    reverse::Bool=false  # TODO
end

function Base.isequal(a::Telomere, b)
    return typeof(a) == typeof(b)
end

function Base.show(t::Telomere)
    return "."
end

# x = Telomere()
# show(x)

In [37]:
"""
Representation for DNA fragment (a gene) in DCJ model.

    Parameters
    ----------
    id: int (required)
        unique id

    dna: str
        What dna fragment does this gene represent?

    reverse: boolean
        Is the DNA fragment reversed?
"""
@with_kw struct Gene <: AbstractGene
    id::Int
    dna::String=""
    reverse::Bool=false
end

function Base.isequal(a::Gene, b::Gene)
    return typeof(a) == typeof(b) && a.id == b.id
end

function Base.show(m::Gene)
    if m.reverse
        print("-" * string(m.id))
    else
        print(string(m.ud))
    end
end

# y = Gene(1, "d", true)
# show(y)

In [49]:
 """The abstraction of a chromosome in DCJ model, a list of telomeres and genes

    Parameters
    ----------
    genes: list of genes or string (req)
        The object to be converted into a circular or linear chromosome.

    """
mutable struct Chromosome
    genes::Vector{AbstractGene}
end

function Chromosome(genes::String, id_counter::Ref{Int64}, id_to_char::Dict{Int, Char}, char_to_id::Dict{Char, Int64}, target::Bool)
    content = AbstractGene[]

    if !((genes[1] == '.') == (genes[end] == '.'))
        throw(ArgumentError("Linear Chromosome must start and end with telomeres."))
    end

    for (i, gene) in pairs(genes)
        dna = string(gene)
        rev = isuppercase(gene)
        telomere = gene == '.'        
        if telomere 
            push!(content, Telomere())
        else  # gene 
            id = id_counter[]

            if target  
                id_to_char[id] = gene
                if gene in keys(char_to_id)
                    push!(char_to_id[lowercase(gene)], id)
                else 
                    char_to_id[lowercase(gene)] = id
                end 
                id_counter[] += 1

            else  # source str 
                ## assuming no duplicates
                id = char_to_id[lowercase(gene)]
            end
            push!(content, Gene(id, dna, rev))
        end 
    end    
    
    # print(content)
    return Chromosome(content)
end

Chromosome

In [9]:
# Genome
mutable struct Genome
    data::Vector{Chromosome}
end

In [63]:
function genome_to_string(A)
end


function string_to_genome(s, id_counter, id_to_char, char_to_id, target)
     """
    Converts string to type Genome 
    """

    chrom_list = Vector{String}()

    chrom = ""
    new_chrom = true 
    linear = false

    for (i, g) in pairs(s)
        # if creating a new chromosome 
        if new_chrom 
            if chrom == ""
                linear = g == '.'
            else  # chrom isn't empty 
                linear = chrom[1] == '.'

                # check if empty chromosome .. 
                if g == '.' 
                    chrom = chrom * g 
                    push!(chrom_list, chrom)
                    chrom = ""
                    continue 
                end 
            end 
            
            chrom = chrom * g
            new_chrom = false  
        
        # appending more genes to chromosome 
        else  
            if linear  
                chrom = chrom * g 
                if g == '.' || i == length(s)  # find another telomere or end of str
                    push!(chrom_list, chrom) 
                    chrom = ""
                    new_chrom = true 
                end 
            else  # circular
                if g =='.'  # close circular chrom (won't incude telomere in chrom)
                    push!(chrom_list, chrom)
                    chrom = "."
                    new_chrom = true 
                elseif i == length(s)  # reach end of str 
                    chrom = chrom * g
                    push!(chrom_list, chrom)
                    chrom = "."
                    new_chrom = true 
                else 
                    chrom = chrom * g
                end 
            end 
        end 
    end

    print(chrom_list)

    gene_list = Chromosome[]

    for c in chrom_list
        chrom = Chromosome(c, id_counter, id_to_char, char_to_id, target)
        push!(gene_list, chrom)
    end 
    
    return Genome(gene_list)
end 

string_to_genome (generic function with 1 method)

In [ ]:
"""
Gene head or tail end. Used in adjacencies
"""
@with_kw struct GeneEnd
    gene::AbstractGene
    head::Bool=true
end

"""Adjacency Data Structure"""
mutable struct Adjacency 
    left_end_gene::GeneEnd
    right_end_gene::GeneEnd
    label::Vector{Gene}

    # Adjacency(left_end_gene::GeneEnd = nothing, right_end_gene::GeneEnd = nothing; label::Vector{Gene} = nothing) = new([left_end_gene, right_end_gene], left_end_gene, right_end_gene, label)
end

function Base.isequal(a::GeneEnd, b::GeneEnd)
    return a.gene == b.gene && a.head == b.head
end

function Base.show(me::GeneEnd)
    if me.head
        print(string(me.gene))
    else
        print(string(me.gene) * "*")
    end
end

# function other_adjacency_end(me::GeneEnd, adj::Adjacency)
#     if adj.left_end_gene == me
#         return adj.right_end_gene
#     else
#         return adj.left_end_gene
#     end
# end

In [67]:
function AdjacencyGraph(src::Genome, target::Genome)
    # store genome as tables (not explicitly creating the adj graph)
    src_location_to_adj = Dict{Int, Vector{GeneEnd}}() 
    src_gene_to_location = Dict{Gene, Vector{Int}}()

    print(src)





    # target_location_to_adj = Dict{Int, Vector{GeneEnd}}() 
    # target_gene_to_location = Dict{Gene, Vector{Int}}()



    # for each adj {p, q} in src genome 
        # create edge connecting {p, q} and vertex of genome B that contains p 
        # create edge connecting {p, q} and vertex of genome B that contains q


    # for each telomere {p} in src genome 
        # create edge connecting {p} and the vertex of target that contains p



end 

AdjacencyGraph()

AdjacencyGraph (generic function with 1 method)

In [69]:
"""
genomes A and B have the same letters of the same multiplicity; telomeres don't matter 
"""

function calculate_distance(src::String, target::String)
    # check_conditions(src, target)

    id_counter = Ref{Int}(1)
    id_to_char = Dict{Int, Char}()
    char_to_id = Dict{Char, Int}()

    target_genome = string_to_genome(target, id_counter, id_to_char, char_to_id, true)
    src_genome = string_to_genome(src, id_counter, id_to_char, char_to_id, false)

    ag = AdjacencyGraph(src_genome, target_genome)
    # dcj_distance = length(ag.commonGenes) - ag.cycles - ag.ab_paths / 2
end

src = ".abcd."
target = ".aCBd."
calculate_distance(src, target)

[".aCBd."][".abcd."]

LoadError: UndefVarError: `GeneEnd` not defined